In [3]:
%pip install tellurium
import matplotlib.pyplot as plt
import tellurium as te
import numpy as np
import time
import math
print('OK')

In [4]:
#--------------------------- CONFIGURACION DE LA LIBRETA ---------------------------
semilla = 123456 # Semilla para la aleatoridad del integrador Gillespie
n = 10 # Intervalo de tiempo a mostrar (en horas)
diferencia = 0.5 # % de diferencia respecto al valor inicial
especie = 'DPGA'
#--------------------------- --------------------------- ---------------------------

In [5]:
# CARGA DE FICHERO
tLoad_0 = time.time()
r = te.loadAntimonyModel('Arnold2011_Zhu2009_Modelo_Estocastico.txt') # Modelo del ciclo de Calvin-Benson (Zhu2009)
print('Fichero cargado en '+ str(time.time() - tLoad_0) +' s')

In [6]:
# Configuracion del integrador Gillespie -> Simulacion Estocastica
r.integrator = 'gillespie'
r.integrator.seed = semilla
r.integrator.variable_step_size = False 
r.integrator.nonnegative = True
print(r.integrator)

In [20]:
species = [especie]
results = [] 
#--------------------------- --------------------------- ---------------------------
print('Realizando simulaciones, espere por favor...')
tSim_0 = time.time()
r.reset() # Reset de seguridad para establecer la simulacion con los valores iniciales

s = r.simulate(0,n,100*n) # Simulacion con 100n puntos en el intervalo de tiempo [0,n]

print('Simulacion realizada en '+ str(time.time() - tSim_0) +' s')
r.plot(s, show=False, alpha=0.7)
te.show() #Para mostrar la grafica - En caso de usar el cuaderno de Tellurium esta es interactiva

In [22]:
valorInicial = r.getGlobalParameterValues();
ID = list(enumerate([x for x in r.getGlobalParameterIds() if ("K" in x or "k" in x)])); # Obtenemos y mostramos los paramatros kn del modelo
print(valorInicial)
print(ID)

Especie (por defecto DPGA) variando el valor de k7 (0.1,**0,2**,0.03)

In [24]:
#--------------------------- PRUEBA GRAFICA INDIVIDUAL ----------------------------
Kn = 7 #ID del parametro k (1-7)
#--------------------------- --------------------------- --------------------------
plt.rcParams["figure.figsize"] = (12, 8)
plt.grid(color='black', linestyle='-', linewidth=2, alpha=0.1)

def simulaIncertidumbre(modelo, valor, parametro):
    dif = diferencia # % de diferencia respecto al valor inicial
    print(parametro,' -> (',(1-dif)*valor,',',valor,',',(1+dif)*valor,')')
    vals = np.linspace((1-dif)*valor, (1+dif)*valor, 3)
    for specie in species:
        r.resetAll()
        for val in vals:
            r.resetToOrigin() #Devolver parametros al estado original
            exec("r.%s = %f" % (parametro,val)) #Ejecuta la instrucción que asigna el valor a la variable k
            r.integrator.seed = semilla
            result = r.simulate(0,n,10*n, selections = ['time', specie])
            plt.plot(result[:,0],result[:,1],label=val)
            plt.title("incertidumbre en " + parametro)
            
        plt.legend()
        plt.xlabel("Tiempo (seg)")
        plt.ylabel("Nº particulas")
        
siguienteParam = ID[Kn-1]
simulaIncertidumbre(r,valorInicial[siguienteParam[0]+1], siguienteParam[1])

In [25]:
#---------------------- CONFIGURACION GRAFICAS ----------------------
plt.grid(color='black', linestyle='-', linewidth=2, alpha=0.1)
plt.subplots_adjust(left=0.1, bottom=0.1, right=0.9, top=0.9, wspace=0.4, hspace=0.6) 
#--------------------------------------------------------------------
def simulaIncertidumbre(modelo, valor, parametro):
    dif = diferencia # % de diferencia respecto al valor inicial
    print(parametro,' -> (',(1-dif)*valor,',',valor,',',(1+dif)*valor,')')
    vals = np.linspace((1-dif)*valor, (1+dif)*valor, 3)
    for specie in species:
        r.resetAll() #Reinicio del modelo
        for val in vals:
            r.resetToOrigin() #Devolver parametros al estado original
            exec("r.%s = %f" % (parametro,val)) #Ejecuta la instrucción que asigna el valor a la variable k
            r.integrator.seed = semilla
            result = r.simulate(0,n,10*n, selections = ['time', specie])
            plt.plot(result[:,0],result[:,1],label=val)
            plt.title("incertidumbre en " + parametro)
            
        plt.legend()
        plt.xlabel("Tiempo (seg)")
        plt.ylabel("Nº particulas")
#--------------------------------------------------------------------
n = len(ID) + 1;
dim = math.ceil(math.sqrt(n))
for i,siguienteParam in enumerate(ID):
    plt.subplot(dim,dim,i+1)
    simulaIncertidumbre(r,valorInicial[siguienteParam[0]+1], siguienteParam[1])